In [2]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

from pathlib import Path

In [3]:
FOLDER = Path("../../data/")

In [4]:
JANUAR_FILE = "yellow_tripdata_2022-01.parquet"
FEBRUAR_FILE = "yellow_tripdata_2022-02.parquet"

In [5]:
fdata = jdata = pd.read_parquet(FOLDER / FEBRUAR_FILE)
fdata["tpep_pickup_datetime"] = pd.to_datetime(fdata["tpep_pickup_datetime"])
fdata["tpep_dropoff_datetime"] = pd.to_datetime(fdata["tpep_dropoff_datetime"])

fdata["duration"] = (jdata["tpep_dropoff_datetime"] - jdata["tpep_pickup_datetime"]).map(lambda x: x.total_seconds() / 60)

In [6]:
#19 Columns
jdata = pd.read_parquet(FOLDER / JANUAR_FILE)
print(jdata.shape)
jdata["tpep_pickup_datetime"] = pd.to_datetime(jdata["tpep_pickup_datetime"])
jdata["tpep_dropoff_datetime"] = pd.to_datetime(jdata["tpep_dropoff_datetime"])

jdata["duration"] = (jdata["tpep_dropoff_datetime"] - jdata["tpep_pickup_datetime"]).map(lambda x: x.total_seconds() / 60)
jdata.head()

(2463931, 19)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0,17.816667
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0,8.400000
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0,8.966667
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0,10.033333
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0,37.533333


In [7]:
jdata.duration.describe()

count    2.463931e+06
mean     1.421220e+01
std      4.644531e+01
min     -3.442400e+03
25%      6.316667e+00
50%      1.018333e+01
75%      1.616667e+01
max      8.513183e+03
Name: duration, dtype: float64

In [8]:
((jdata.duration >=1) & (jdata.duration<=60)).mean()

0.9827547930522406

In [23]:
jdata = jdata[((jdata.duration >=1) & (jdata.duration<=60))]
fdata = fdata[((fdata.duration >=1) & (fdata.duration<=60))]

In [11]:
cat_values = ["PULocationID", "DOLocationID"]
dv = DictVectorizer()

train_dict = jdata[cat_values].astype(str).to_dict(orient='records')
train_data = dv.fit_transform(train_dict)

In [13]:
len(dv.feature_names_)

515

In [16]:
target  = "duration"
lm = LinearRegression()
lm.fit(train_data, jdata[target])

LinearRegression()

In [18]:
mean_squared_error(jdata[target], lm.predict(train_data), squared=False)

6.98619083368594

In [24]:
test_dict = fdata[cat_values].astype(str).to_dict(orient='records')
test_data = dv.transform(test_dict)

In [25]:
mean_squared_error(fdata[target], lm.predict(test_data), squared=False)

7.786406780434021